Reference: https://github.com/simongeek/KerasVGGcifar10/blob/master/vggkeras.py

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

from datetime import timedelta
from PIL import Image
import sklearn

from keras import callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.layers import Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_normal
from keras.utils import np_utils
# from keras_sequential_ascii import sequential_model_to_ascii_printout
# from keras import backend as K
# if K.backend()=='tensorflow':
#     K.set_image_dim_ordering("th")

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()
tf.config.list_logical_devices()

In [3]:
# gpu_options = tf.GPUOptions(allow_growth=True)
# session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [4]:
images_dir = 'images'
cwd = os.getcwd()

In [5]:
train_csv = pd.read_csv(os.path.join(cwd,'train.csv'))
test_csv = pd.read_csv(os.path.join(cwd,'test.csv'))

In [6]:
os.path.join(cwd,'images','train','emergency')

'/home/jupyter/Learnings/images/train/emergency'

In [1]:
if not os.path.isfile(os.path.join(cwd,'Data','train','emergency')):
    os.mkdir(os.path.join(cwd,'Data','train','emergency'))
    os.mkdir(os.path.join(cwd,'Data','train','not_emergency'))
#Move Images to class wise folders
for ind,row in train_csv.iterrows():
    if row['emergency_or_not']==0:
        img = row['image_names']
        shutil.copy(os.path.join(cwd,'images',img), os.path.join(cwd,'Data','train','not_emergency'))
    else:
        img = row['image_names']
        shutil.copy(os.path.join(cwd,'images',img), os.path.join(cwd,'Data','train','emergency'))    

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
#         brightness_range=[0.2,0,5],       
        channel_shift_range=0.4,
        cval=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',
        validation_split=0.2)

In [18]:
train_generator = datagen.flow_from_directory(
    os.path.join(cwd,'Data','train'),
    target_size=(224, 224),
    batch_size=2,
    class_mode='binary')

Found 1647 images belonging to 2 classes.


In [19]:
batch_size= 2
num_classes = 2
epochs = 5
BATCH_NORM = True

# VGG16 From Scratch

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(224,224,3), name='block1_conv1'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', name='block1_conv2'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

model.add(Conv2D(128, (3, 3), padding='same', name='block2_conv1'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', name='block2_conv2'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

model.add(Conv2D(256, (3, 3), padding='same', name='block3_conv1'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same', name='block3_conv2'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same', name='block3_conv3'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same', name='block3_conv4'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

model.add(Conv2D(512, (3, 3), padding='same', name='block4_conv1'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', name='block4_conv2'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', name='block4_conv3'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', name='block4_conv4'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

model.add(Conv2D(512, (3, 3), padding='same', name='block5_conv1'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', name='block5_conv2'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', name='block5_conv3'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', name='block5_conv4'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))

# model.add(Flatten())

model.add(Dense(1024))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096, name='fc2'))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(BatchNormalization()) if BATCH_NORM else None
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.0005, decay=0, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [ ]:
history = model.fit(train_generator,
                    epochs=10)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.xlabel('Epoch')
plt.ylabel('Categorical_CrossEntropy')
plt.title('Loss per epochs')
plt.legend(['Train','Valid'])

In [ ]:
test_image = []
for im in tqdm(test_csv['image_names'].to_list()):
    img = image.load_img(os.path.join(os.path.join(cwd,'images'),im))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

In [ ]:
prediction = model.predict_classes(test)
np.unique(prediction)

In [ ]:
test_csv['emergency_or_not'] = prediction
test_csv.to_csv('submission.csv',index=False)